In [50]:
# import dependencies
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import numpy as np
import sqlite3
import os
from datetime import datetime as dt

In [27]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# define the url and visit it with browser
freebird_url = "http://www.gasnom.com/ip/freebird/oauc.cfm?type=1"
browser.visit(freebird_url)

In [108]:
# grab the table data
tables = pd.read_html(freebird_url)
freebird_info = tables[0]
freebird_raw = tables[2]

In [109]:
# reformat the table data
tsp_name = freebird_info.iloc[0][1]
posting_date = freebird_info.iloc[2][1]

del freebird_raw[12]
freebird_raw.columns = ["location_name","loc","loc_purp_desc","loc_qti","flow_ind","all_qty_avail",\
                       "design_capacity", "operating_capacity","tsq","oac","it_indicator","qty_reason"]
freebird_raw = freebird_raw.fillna(0)
freebird_df = freebird_raw.loc[freebird_raw['location_name'] != 0]
freebird_df = freebird_df.drop(freebird_df.index[0])
freebird_df = freebird_df.reset_index(drop=True)

freebird_df.insert(0, 'date', posting_date)
freebird_df.insert(1, 'tsp_name', tsp_name)
freebird_df

,date,tsp_name,location_name,loc,loc_purp_desc,loc_qti,flow_ind,all_qty_avail,design_capacity,operating_capacity,tsq,oac,it_indicator,qty_reason
0,"October 8, 2020 05:35 PM CT",Freebird Gas Storage L.L.C.,NWAGD,0002,Storage Area,STQ,BD,Y,25000,25000,0,25000,0,0
1,"October 8, 2020 05:35 PM CT",Freebird Gas Storage L.L.C.,Tennessee_PL_Inj,021039,Storage Area,STQ,BD,Y,250000,110000,4100,105900,0,0
2,"October 8, 2020 05:35 PM CT",Freebird Gas Storage L.L.C.,Tennessee_PL_With,012709,Storage Area,STQ,BD,Y,250000,100000,-5649,94351,0,0
